In [1]:
from flask import Flask, render_template, request, jsonify
import pickle
from twilio.rest import Client
import numpy as np
from scipy.stats import zscore
import matplotlib.pyplot as plt

In [2]:
app = Flask(__name__)

In [3]:
# Twilio credentials
account_sid = 'ACb152e1f3a2b891f4f9f2f35172543b22'
auth_token = 'f17e684c2b35a4adfc62c83356d30baf'
twilio_phone_number = '+12673769611'
user_phone_number = '+918168594867'

client = Client(account_sid, auth_token)

In [4]:
def Z_Score_Value(Value):
    mean = 158668.18606839626
    std = 264941.5785563747
    Z_score = (Value - mean)/std
    Threshold_value = 12.40
    
    if Z_score >= Threshold_value:
        return 1
    else:
        return 0
    
def Tukey_Fences_Values(Value):
    Q1 = 12149.490000000002
    Q3 = 213762.15000000002
    IQR = Q3 - Q1
    tukey_threshold = 16.00 * IQR
    
    if Value < Q1 - tukey_threshold or  Value > Q3 + tukey_threshold:
        return 1
    else:
        return 0
    
    
def modified_zscore_values(value):
    median = 76345.78
    median_absolute_deviation = np.median(np.abs(value - median))
    modified_z_score = np.abs(0.6745 * (value - median) / median_absolute_deviation)
    threshold = 32.30
    
    if modified_z_score > threshold:
        return 1
    else:
        return 0

In [5]:
@app.route('/')
def index():
    return render_template('index.html')

In [6]:
@app.route('/predict_anomaly', methods=['POST'])
def predict_anomaly():
    try:
        selected_function = request.json['selectedFunction']
        payment_type = request.json['paymentType']
        old_balance = request.json['oldBalance']
        amount = request.json['amount']

        # Choose the selected function
        if selected_function == 'modified_zscore':
            prediction = modified_zscore_values(amount)
        elif selected_function == 'tukey_fences':
            prediction = Tukey_Fences_Values(old_balance)
        elif selected_function == 'z_score':
            prediction = Z_Score_Value(amount)
        else:
            raise ValueError('Invalid function selected')

        # Simulate updating the balance (replace with your actual logic)
        current_balance = old_balance - amount if payment_type == 'debit' else old_balance + amount
        
         # Send SMS if prediction is 1 (anomaly)
        if prediction == 1:
            send_sms(f'Fraud detected! Payment Type: {payment_type}, Old Balance: {old_balance}, Amount: {amount}')

        return jsonify({'prediction': prediction, 'currentBalance': current_balance})

    except Exception as e:
        return jsonify({'error': str(e)})
    

def send_sms(message):
    try:
        message = client.messages.create(
            to=user_phone_number,
            from_=twilio_phone_number,
            body=message
        )
        print(f"Message sent successfully. SID: {message.sid}")
    except Exception as e:
        print(f"Error sending SMS: {e}")

In [7]:
with open('xgb_model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)
    

c:\Users\visha\anaconda3\lib\site-packages\xgboost\core.py:160: UserWarning: [22:49:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)


In [8]:
@app.route('/machinelearning')
def machinelearning():
    return render_template('machinelearning.html')

@app.route('/machinelearningpredict', methods=['POST'])
def machinelearningpredict():
    type_of_payment = request.form['type_of_payment']
    old_balance = float(request.form['old_balance'])
    amount = float(request.form['amount'])

    # Map type_of_payment to encoded values
    payment_mapping = {'cash_out': 1, 'payment': 3, 'cash_in': 0, 'transfer': 4, 'debit': 2}
    encoded_payment = payment_mapping.get(type_of_payment.lower(), 0)

    # Create new_balance variable
    new_balance = old_balance + amount

    # Make prediction
    prediction = model.predict([[encoded_payment, amount, old_balance, new_balance]])

    # Display the result on the HTML page
    if prediction[0] == 0:
        result = 'Not Fraud'
    else:
        result = 'Fraud Transaction'
        send_sms(f'Fraud detected! Payment Type: {type_of_payment}, Old Balance: {old_balance}, Amount: {amount}')
    return render_template('machinelearning.html', result=result)

def send_sms(message):
    try:
        message = client.messages.create(
            to=user_phone_number,
            from_=twilio_phone_number,
            body=message
        )
        print(f"Message sent successfully. SID: {message.sid}")
    except Exception as e:
        print(f"Error sending SMS: {e}")

In [9]:
if __name__ == '__main__':
    app.run(host="0.0.0.0")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.20.10.2:5000/ (Press CTRL+C to quit)
172.20.10.2 - - [02/Jun/2024 22:49:29] "GET / HTTP/1.1" 200 -
172.20.10.2 - - [02/Jun/2024 22:49:29] "GET /favicon.ico HTTP/1.1" 404 -
172.20.10.2 - - [02/Jun/2024 22:49:53] "POST /predict_anomaly HTTP/1.1" 200 -


Message sent successfully. SID: SMe93b7668b1d4f974a5aab373b95fce11


172.20.10.2 - - [03/Jun/2024 10:05:14] "POST /predict_anomaly HTTP/1.1" 200 -
172.20.10.2 - - [03/Jun/2024 10:05:33] "POST /predict_anomaly HTTP/1.1" 200 -
172.20.10.2 - - [03/Jun/2024 10:05:33] "POST /predict_anomaly HTTP/1.1" 200 -


Message sent successfully. SID: SM0035e2ca271f720e475ce668c92f4854
Message sent successfully. SID: SMd650b988e60df78c2dd57c2a8cc2a88e


172.20.10.2 - - [03/Jun/2024 10:05:38] "POST /predict_anomaly HTTP/1.1" 200 -


Message sent successfully. SID: SM01ac6e524a4651562f7eaa069a1b68b7


172.20.10.2 - - [03/Jun/2024 10:05:59] "POST /predict_anomaly HTTP/1.1" 200 -


Message sent successfully. SID: SM3afb6d0a82822a4ab96f89b4c200db65
